<a href="https://colab.research.google.com/github/Aditya-Shandilya1182/NLP_RNN/blob/main/NLP_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-cdb8f7dd-cf24-d6e7-7c82-f9ca4bec53f5)


In [5]:
import tensorflow as tf

# Data

Here I have used Real or Not? dataset from kaggle. The real tweets in this dataset are baout disasters and not real tweets are not about disasters.

[Dataset Link](https://www.kaggle.com/c/nlp-getting-started/data)

In [6]:
import pandas as pd
train_df = pd.read_csv("/content/drive/MyDrive/train.csv")
test_df = pd.read_csv("/content/drive/MyDrive/test.csv")
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [7]:
from sklearn.utils import shuffle
train_df_shuffled = shuffle(train_df, random_state=42)

In [8]:
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [9]:
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [10]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(train_df_shuffled["text"].to_numpy(), train_df_shuffled["target"].to_numpy(),
                                                    test_size=0.1, random_state=42)

In [11]:
len(X_train), len(y_train), len(X_test), len(y_test)

(6851, 6851, 762, 762)

In [12]:
X_train[:10], y_train[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object),
 array([0,

# Text Vectorization

In [13]:
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization

max_vocab_length = 10000
max_length = 15
text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                    output_mode="int",
                                    output_sequence_length=max_length)

In [14]:
text_vectorizer.adapt(X_train)

In [15]:
example = "The football team is practising for tommorow's game."
text_vectorizer([example])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[   2, 1523,  697,    9,    1,   10,    1,  397,    0,    0,    0,
           0,    0,    0,    0]])>

# Embedding Layer

In [16]:
tf.random.set_seed(42)

embedding = tf.keras.layers.Embedding(input_dim=max_vocab_length,
                                      output_dim=128,
                                      embeddings_initializer="uniform",
                                      input_length=max_length,
                                      name="embedding_0")
embedding

In [17]:
embedding_example = embedding(text_vectorizer([example]))
embedding_example

<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[-0.0323771 , -0.01761147, -0.02716299, ...,  0.0253708 ,
         -0.00501593,  0.04764794],
        [-0.04409988, -0.01381069, -0.03661665, ...,  0.02104126,
         -0.00692949, -0.00883362],
        [ 0.03090444,  0.03477449,  0.0386092 , ..., -0.01210294,
         -0.0345455 , -0.04793049],
        ...,
        [ 0.00467576, -0.00381295,  0.04139495, ...,  0.01566341,
         -0.02112172,  0.03749749],
        [ 0.00467576, -0.00381295,  0.04139495, ...,  0.01566341,
         -0.02112172,  0.03749749],
        [ 0.00467576, -0.00381295,  0.04139495, ...,  0.01566341,
         -0.02112172,  0.03749749]]], dtype=float32)>

# LSTM Model

In [18]:
tf.random.set_seed(42)

LSTM_model_embedding = tf.keras.layers.Embedding(input_dim=max_vocab_length,
                                                  output_dim=128,
                                                  embeddings_initializer="uniform",
                                                  input_length=max_length,
                                                  name="LSTM_embedding")
inputs = tf.keras.layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = LSTM_model_embedding(x)
x = tf.keras.layers.LSTM(64)(x)
outputs = tf.keras.layers.Dense(1, activation="sigmoid")(x)
LSTM_model = tf.keras.Model(inputs, outputs, name="model_LSTM")

In [19]:
LSTM_model.compile(loss="binary_crossentropy",
                    optimizer=tf.keras.optimizers.Adam(),
                    metrics=["accuracy"])

In [20]:
LSTM_model.summary()

Model: "model_LSTM"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVe  (None, 15)                0         
 ctorization)                                                    
                                                                 
 LSTM_embedding (Embedding)  (None, 15, 128)           1280000   
                                                                 
 lstm (LSTM)                 (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 1)                 65        
                                                                 
Total params: 1329473 (5.07 MB)
Trainable params: 1329473 (5.07 MB)
Non-trainable params: 0 (0.00 Byte)
__________________

In [21]:
LSTM_model_history = LSTM_model.fit(X_train,
                                    y_train,
                                    epochs = 5,
                                    validation_data = (X_test, y_test))

Epoch 1/5
215/215 [==============================] - 10s 31ms/step - loss: 0.5119 - accuracy: 0.7459 - val_loss: 0.4531 - val_accuracy: 0.7874
Epoch 2/5
215/215 [==============================] - 3s 13ms/step - loss: 0.3183 - accuracy: 0.8707 - val_loss: 0.5116 - val_accuracy: 0.7848
Epoch 3/5
215/215 [==============================] - 2s 11ms/step - loss: 0.2210 - accuracy: 0.9158 - val_loss: 0.5718 - val_accuracy: 0.7730
Epoch 4/5
215/215 [==============================] - 2s 7ms/step - loss: 0.1600 - accuracy: 0.9442 - val_loss: 0.5952 - val_accuracy: 0.7835
Epoch 5/5
215/215 [==============================] - 2s 8ms/step - loss: 0.1145 - accuracy: 0.9590 - val_loss: 0.8528 - val_accuracy: 0.7546


In [22]:
LSTM_model_preds = LSTM_model.predict(X_test)
LSTM_model_preds.shape, LSTM_model_preds[:10]

24/24 [==============================] - 0s 3ms/step


((762, 1),
 array([[0.02156409],
        [0.77112234],
        [0.9990206 ],
        [0.05385847],
        [0.00332098],
        [0.9988102 ],
        [0.806177  ],
        [0.99938405],
        [0.9992066 ],
        [0.6365114 ]], dtype=float32))

In [23]:
#Rounding up the above probabilities into integers(1 or 0)
LSTM_preds = tf.squeeze(tf.round(LSTM_model_preds))
LSTM_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [24]:
from sklearn.metrics import accuracy_score, precision_score

LSTM_accuracy = accuracy_score(y_test, LSTM_preds)
LSTM_precision = precision_score(y_test, LSTM_preds)

print(f'Accuracy: {LSTM_accuracy:.2f}')
print(f'Precision: {LSTM_precision:.2f}')

Accuracy: 0.75
Precision: 0.75


# Bidirectonal RNN model

In [25]:
tf.random.set_seed(42)
model_embedding = tf.keras.layers.Embedding(input_dim=max_vocab_length,
                                            output_dim=128,
                                            embeddings_initializer="uniform",
                                            input_length=max_length,
                                            name="embedding_bidirectional")
inputs = tf.keras.layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = model_embedding(x)
x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64))(x)
outputs = tf.keras.layers.Dense(1, activation="sigmoid")(x)
bidirectonal_model = tf.keras.Model(inputs, outputs, name="model_bidirectional")

In [26]:
bidirectonal_model.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [27]:
bidirectonal_model.summary()

Model: "model_bidirectional"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVe  (None, 15)                0         
 ctorization)                                                    
                                                                 
 embedding_bidirectional (E  (None, 15, 128)           1280000   
 mbedding)                                                       
                                                                 
 bidirectional (Bidirection  (None, 128)               98816     
 al)                                                             
                                                                 
 dense_1 (Dense)             (None, 1)                 129       
                                               

In [28]:
bidirectonal_model_history = bidirectonal_model.fit(X_train,
                                                    y_train,
                                                    epochs = 5,
                                                    validation_data = (X_test, y_test))

Epoch 1/5
215/215 [==============================] - 12s 35ms/step - loss: 0.5095 - accuracy: 0.7441 - val_loss: 0.4587 - val_accuracy: 0.7822
Epoch 2/5
215/215 [==============================] - 2s 10ms/step - loss: 0.3136 - accuracy: 0.8724 - val_loss: 0.5223 - val_accuracy: 0.7703
Epoch 3/5
215/215 [==============================] - 3s 12ms/step - loss: 0.2130 - accuracy: 0.9181 - val_loss: 0.5737 - val_accuracy: 0.7651
Epoch 4/5
215/215 [==============================] - 3s 13ms/step - loss: 0.1470 - accuracy: 0.9501 - val_loss: 0.6680 - val_accuracy: 0.7664
Epoch 5/5
215/215 [==============================] - 2s 10ms/step - loss: 0.1036 - accuracy: 0.9651 - val_loss: 0.6885 - val_accuracy: 0.7756


In [29]:
bidirectonal_model_preds_probs = bidirectonal_model.predict(X_test)

24/24 [==============================] - 1s 3ms/step


In [30]:
bidirectonal_model_preds_probs[:10]

array([[0.06622979],
       [0.87450325],
       [0.9993411 ],
       [0.15781862],
       [0.00461429],
       [0.99710256],
       [0.9662969 ],
       [0.99957865],
       [0.9997209 ],
       [0.33471277]], dtype=float32)

In [31]:
bidirectonal_model_preds = tf.squeeze(tf.round(bidirectonal_model_preds_probs))

In [32]:
bidirectonal_model_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [33]:
from sklearn.metrics import accuracy_score, precision_score

bidirectonal_model_accuracy = accuracy_score(y_test, bidirectonal_model_preds)
bidirectonal_model_precision = precision_score(y_test, bidirectonal_model_preds)

print(f'Accuracy: {bidirectonal_model_accuracy:.2f}')
print(f'Precision: {bidirectonal_model_precision:.2f}')

Accuracy: 0.78
Precision: 0.78
